# Gradientes de balance de masa (MBG) y su influencia en el flujo glaciar

En el cuaderno de introducción, mencionamos brevemente el gradiente de balance de masa de un glaciar. Esto es lo que vamos a echar un vistazo más de cerca ahora.
Si el concepto de balance de masa es completamente nuevo para usted, lea brevemente sobre él [aqui](http://www.antarcticglaciers.org/glacier-processes/introduction-glacier-mass-balance/), hasta el párrafo "*Entonces, ¿qué es el balance de masa de glaciares?*".
En este cuaderno, configuraremos algunos glaciares simples para explorar más a fondo qué características de los glaciares cambian con diferentes gradientes de balance de masa. También veremos los temas de volumen y tiempo de respuesta.

Objetivos de este cuaderno:
- El estudiante será capaz de explicar los términos:
- Gradiente de balance de masa,
- Estado de equilibrio
- Tiempo de respuesta.

Primero, tenemos que importar las clases relevantes

In [ ]:
from oggm_edu import MassBalance, GlacierBed, Glacier, GlacierCollection

## Inicializando el glaciar
Comenzamos configurando un glaciar simple con un lecho rocoso lineal (ver [edu_intro](edu_intro_es.ipynb)) para generar un punto de partida para nuestro experimento.

In [ ]:
# Bed
bed = GlacierBed(top=3400, bottom=1500, width=400)
# Mass balance
mass_balance = MassBalance(ela=3000, gradient=4)
# Glacier
glacier = Glacier(bed=bed, mass_balance=mass_balance)

## Cambiando el gradiente de balance de masa (MBG)
El MBG se define como el cambio del balance de masa con altitud [¹](#References). Depende en gran medida del clima en el glaciar [¹](#References).
Echemos un vistazo a los efectos del MBG creando algunos glaciares con diferentes gradientes. En el cuaderno de introducción, nuestro glaciar tenía una pendiente de 4 mm/m, así que agreguemos un glaciar con una pendiente más débil y otro con una pendiente más fuerte.
Volveremos a hacer uso de `GlacierCollection` para progresar rápidamente y visualizar los glaciares.

In [ ]:
collection = GlacierCollection()
# Fill the collection. This time we change the attributes on creation.
collection.fill(glacier, n=3, attributes_to_change=
                {'gradient':[0.3, 4, 15]}
               )

### Creación de un perfil de balance de masa complejo
También es posible crear un perfil de balance de masa más complejo.
Esto se logra pasando una lista de gradientes y otra lista con la altitud de los puntos de ruptura entre ellos.

In [ ]:
# Add complex mb profile
complex_mb = MassBalance(ela=3000, gradient=[3, 7, 10, 15],
                         breakpoints=[2700, 2250, 1800])
glacier_complex_mb = Glacier(bed=bed, mass_balance=complex_mb)

In [ ]:
# Add the new glacier to the collection.
collection.add(glacier_complex_mb)

In [ ]:
# Progress collection to the same year.
collection.progress_to_year(300)

In [ ]:
collection.plot()

In [ ]:
collection.plot_mass_balance()

Un gradiente de balance de masa más fuerte (línea más plana en el gráfico anterior) implica un cambio mayor del balance de masa con la altitud.
Podemos ver esto en el gráfico anterior: el balance de masa anual apenas cambia con la altitud para el glaciar con el gradiente de balance de masa más débil (azul), mientras que hay una diferencia considerable entre el balance de masa anual superior e inferior para el glaciar con la masa más fuerte. gradiente de equilibrio (verde).
Esto a su vez afecta el crecimiento del glaciar.
Un fuerte gradiente de balance de masa implica que se agrega más hielo a la zona de acumulación mientras que se extrae más hielo de la zona de ablación cada año en comparación con un glaciar con un gradiente más débil.
Esto da como resultado un mayor flujo de hielo y, por lo tanto, el glaciar crece más rápido.
Esta es la razón por la cual el glaciar con el gradiente más fuerte exhibe el mayor crecimiento durante nuestros experimentos (Glaciar verde en el gráfico de colección arriba).
<div class="alerta alerta-éxito">
<detalles><summary><b>¿Qué opinas? ¿Dónde encontramos glaciares con altos MBG?</b> <i>Haz clic para obtener más detalles</i></summary>
Encontrará una breve explicación en este [paragraph](http://www.antarcticglaciers.org/modern-glaciers/introduction-glacier-mass-balance/#SECTION_3) en AntarcticGlaciers.org.
</div>

## Estado de equilibrio

Los glaciares cambian su geometría como una forma de adaptarse al clima[³](#References). Si la acumulación aumenta, el glaciar crecerá más abajo del ELA para aumentar la ablación. De manera similar, si la temperatura aumenta y la ablación aumenta, la longitud del glaciar disminuirá.
Si el clima permanece constante durante un tiempo suficiente, los glaciares alcanzarán un estado de equilibrio con su clima, donde la acumulación = ablación [⁴](#References).
Con esto en mente, veremos qué tan rápido nuestros glaciares, con diferentes gradientes, alcanzan este estado y compararemos sus formas:

In [ ]:
# We reset the collection.
collection = GlacierCollection()
# Fill the collection. This time we change the attributes on creation.
collection.fill(glacier, n=3, attributes_to_change=
                {'gradient':[0.3, 4, 15]}
               )

In [ ]:
# First we progress the glaciers to equilibrium.
collection.progress_to_equilibrium()

In [ ]:
# Plot it
collection.plot()

Los diferentes glaciares alcanzan su estado de equilibrio después de un número diferente de años. ¿Qué nos muestra la figura? ¿Qué glaciar es el más grueso y largo? Veamos números específicos:

In [ ]:
collection

El glaciar con el gradiente más fuerte alcanza primero el estado de equilibrio. Este es también el glaciar más grande y más largo.

## Tiempo de respuesta
El tiempo de respuesta del glaciar es el período de tiempo que necesita un glaciar para ajustar su geometría a los cambios en el balance de masa causados ​​por el cambio climático y alcanzar un nuevo estado de equilibrio.
Hay algunas definiciones diferentes para su cálculo, OGGM-Edu usa la definición de Oerlemans (vea la fórmula a continuación) [⁵](#References).
Un glaciar OGGM-Edu tiene un atributo `.response_time` que se calcula en función de la diferencia entre los dos últimos estados de equilibrio en la historia del glaciar.
Hasta ahora solo te hemos mostrado cómo hacer progresar un glaciar hasta el equilibrio, pero no cómo dejarlo y llegar a un segundo.
Este será el primer paso para obtener el tiempo de respuesta de un glaciar.
Usamos el glaciar desde el comienzo del cuaderno.

In [ ]:
# Check the state, everything should be zero/nan here.
glacier

Primero progresamos el glaciar al equilibrio

In [ ]:
glacier.progress_to_equilibrium()

In [ ]:
glacier.plot()

Podemos acceder a los estados de equilibrio almacenados de un glaciar

In [ ]:
glacier.eq_states

Ahora queremos establecer un escenario de cambio climático para el glaciar.
Esto se hace fácilmente con el método `.add_temperature_bias`.
Este método toma un sesgo de temperatura deseado ($+/-$) y una duración, que especifica cuánto tardará el glaciar en alcanzar el estado climático.
Por ejemplo, si establecemos el sesgo en 1 y la duración en 50, el clima tardará 50 años en calentarse 1 grado.
Cuando un glaciar tiene un escenario de cambio climático, los métodos `progress_` funcionarán internamente de manera un poco diferente, pero esto no es algo que notará.
Dado que estamos puramente interesados ​​en la respuesta del glaciar, creamos un escenario de cambio climático con una duración de 1 año.

In [ ]:
# Add a climate change scenario
glacier.add_temperature_bias(bias=1., duration=1)

<div class="alerta alerta-éxito">
<detalles><resumen><b>
¿Cómo responde el ELA si elevamos la temperatura?</b> <i>Haga clic para obtener una pista</i></summary>
El aumento de la temperatura aumentará el tamaño del área de ablación del glaciar y, por lo tanto, aumentará la ELA.
</detalles>
</div>

In [ ]:
# We can then progress the glacier to equilibrium again
glacier.progress_to_equilibrium()

In [ ]:
glacier.plot()

In [ ]:
glacier.eq_states

El siguiente paso es calcular los tiempos de respuesta de nuestros glaciares. Uno podría pensar que es tan simple como mirar los años anteriores y hacer una simple resta. ¡Sin embargo, éste no es el caso! En realidad, la velocidad a la que cambia un glaciar es cada vez menor y nunca se alcanza realmente un estado de equilibrio completo. Debido a esto, el tiempo de respuesta se considera el tiempo que le ha tomado al glaciar completar la mayor parte del ajuste, más específicamente todo menos un factor de $1/e$.
Para modelos numéricos como nuestros glaciares, es común usar [volume response time](https://www.tandfonline.com/doi/pdf/10.1080/00040851.1997.12003238?needAccess=true), de Oerlemans [volume response time](https://www.tandfonline.com/doi/pdf/10.1080/00040851.1997.12003238?needAccess=true):
$$
\tau = t \left(V=V_2 - \frac{V_2 - V_1}{e}\right)
$$
donde $V_1$ y $V_2$ corresponden al volumen del glaciar en el estado de equilibrio inicial y nuevo respectivamente.
Afortunadamente, esto lo hace nuestro objeto glaciar, por lo que solo podemos echar un vistazo al atributo `.response_time`

In [ ]:
glacier.response_time

o simplemente mira la representación

In [ ]:
glacier

Ahora que hemos introducido el concepto de tiempo de respuesta, podemos aplicarlo y ver cómo los diferentes gradientes de balance de masa afectan el tiempo de respuesta.
Para ello necesitamos una nueva colección.

In [ ]:
# Bed
bed = GlacierBed(top=3400, bottom=1500, width=400)
# Mass balance
mass_balance = MassBalance(ela=3000, gradient=4)
# Glacier
glacier = Glacier(bed=bed, mass_balance=mass_balance)

In [ ]:
collection = GlacierCollection()
# Fill the collection. This time we change the attributes on creation.
collection.fill(glacier, n=3, attributes_to_change=
                {'gradient':[1, 4, 15]}
               )

In [ ]:
collection.progress_to_equilibrium()

In [ ]:
collection

Luego tenemos que configurar un escenario de cambio climático para los glaciares en la colección.

In [ ]:
# Same scenario for all the glaciers.
collection.glaciers[0].add_temperature_bias(bias=1., duration=1)
collection.glaciers[1].add_temperature_bias(bias=1., duration=1)
collection.glaciers[2].add_temperature_bias(bias=1., duration=1)

In [ ]:
# Progress to equilibrium again
collection.progress_to_equilibrium()

In [ ]:
collection

In [ ]:
collection.plot_history()

También podemos mirar la historia del estado de uno de los glaciares.

In [ ]:
# Plots the glacier outline for the equilibrium states.
collection.glaciers[0].plot_state_history(eq_states=True)

El glaciar con MBG más débil necesita más tiempo para adaptarse a un cambio climático en comparación con los otros glaciares. Por otro lado, el glaciar con el gradiente más fuerte solo necesita alrededor de dos décadas para ajustar su forma al nuevo clima (Un ejemplo del mundo real: el glaciar Franz Josef en Nueva Zelanda[⁶](#References))[⁶](#References). El tiempo de respuesta de los glaciares con gradientes débiles es en realidad mucho más largo que 200 años, en realidad más cercano a 2000 años.
<div class="alerta alerta-éxito">
<detalles><resumen><b>¿Por qué la MBG cambia el tiempo de respuesta de un glaciar? ¿Qué otros factores, además del MBG, cree que afectan el tiempo de respuesta de un glaciar?</b> <i>Haga clic en mí para obtener una pista</i></summary>
El MBG afecta la rapidez con la que fluirá el glaciar y, a su vez, la rapidez con la que puede cambiar la forma y el tamaño del glaciar. En general, debemos considerar que el tiempo de respuesta también depende de otras características como el tipo de glaciar, el tamaño, la pendiente del fondo y la elevación promedio de la superficie.
</detalles>
</div>


## Referencias
¹ Rasmussen, L. A. y Andreassen, L. M. (2005). Gradientes estacionales de balance de masa en Noruega. *Revista de Glaciología*, 51(175), 601-606.
² Oerlemans, J. y Fortuin, J. P. F. (1992). Sensibilidad de los glaciares y pequeños casquetes polares al efecto invernadero. *Ciencia, 258(5079)*, 115-117.
³Oerlemans, J. (2001). *Glaciares y cambio climático*. Prensa CRC.
⁴ Enciclopedia de nieve, hielo y glaciares, V.P. Singh, P. Singh y U.K. Haritashya, Editores. 2011, Springer: Dordrecht, Países Bajos. pags. 245-256.
⁵ Oerlemans, J. (1997). Sensibilidad climática del glaciar Franz Josef, Nueva Zelanda, según lo revelado por el modelado numérico. *Investigación sobre el Ártico y los Alpes*, 29(2), 233-239.
⁶ Anderson, B., Lawson, W. y Owens, I. (2008). Respuesta del glaciar Franz Josef Ka Roimata o Hine Hukatere al cambio climático. Cambio Global y Planetario, 63(1), 23-30.
⁷ Cuffey, K. M. y Paterson, WSB *La Física de los Glaciares, 4ª edición*, 704 (Academic Press, 2010).


## ¿Que sigue?

xx_markdown_enlace_xx